In [1]:
import sys
sys.path.insert(1, '../scripts')

In [2]:
import numpy as np
import pandas as pd
from Bio import AlignIO
import crutil

## Loading list of dubious, uncharacterized, and verified ORFs

In [3]:
dubious_orfs_df = pd.read_table("../../data/sc_orfs/dubious_orfs.tsv", header=None,
                                names=["DBID", "systematic_name", "organism", "standard_name", "gene_name"])

In [4]:
unchar_orfs_df = pd.read_table("../../data/sc_orfs/uncharacterized_orfs.tsv", header=None,
                               names=["DBID", "systematic_name", "organism", "standard_name", "gene_name"])

In [5]:
verified_orfs_df = pd.read_table("../../data/sc_orfs/verified_orfs.tsv", header=None,
                               names=["DBID", "systematic_name", "organism", "standard_name", "gene_name"])

In [6]:
dubious_orfs = dubious_orfs_df["systematic_name"].tolist()

In [7]:
unchar_orfs = unchar_orfs_df["systematic_name"].tolist()

In [8]:
verified_orfs = verified_orfs_df["systematic_name"].tolist()

In [9]:
bad_orfs = dubious_orfs + unchar_orfs

In [10]:
len(bad_orfs)

1410

## Removing dubious and uncharacterized ORFs

In [11]:
df = pd.read_csv('../../data/charged_regions/cr_raw.csv', comment='#')

In [12]:
df

,orf,gene,seq.len,left.bound,right.bound,region.seq,region.len,charge.asymmetry,frac.charge,kappa1,kappa2,uni_id,orf_label
0,YAL011W,SWC3,626,0,54,MPAVLRTRSKESSIEQKPASRTRTRSRRGKRGRDDDDDDDDEESDD...,55,0.015674,0.527273,0.297167,0.616928,P31376,verified
1,YAL011W,SWC3,626,169,265,RLFILKNDKIEQKWQDEQELKKKEKELKRKNDAEAKRLRMEERKRQ...,97,0.050753,0.536082,0.051716,0.110243,P31376,verified
2,YAL011W,SWC3,626,361,424,KTAATEPEPKKADDENAEKQQSKEAKTTAESTQVDVKKEEEDVKEK...,64,0.007812,0.500000,0.044964,0.096912,P31376,verified
3,YAL011W,SWC3,626,470,525,KSVVEFLEDTDEIIISWIVIHNSKEIEKFKTKKIKAKLKADQKLNK...,56,0.000000,0.428571,0.114622,0.300249,P31376,verified
4,YAL013W,DEP1,406,81,163,TLTESLKRPHEDEKEAIDEAKKMKVPGENEDESKEEEKSQELEEAI...,83,0.141633,0.542169,0.093375,0.239955,P31385,verified
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,YPR163C,TIF3,437,379,433,SVYDVLRTEDDDEDEEAEKQNGDAKENKVDAAVEKLQDKTAQLTVE...,55,0.122909,0.454545,0.115449,0.366767,P34167,verified
1047,YPR169W,JIP5,493,434,491,LVGLSKEELLDELDKDLKEDHQEEKESNSKSVKKRKIMKENNKKKD...,58,0.000556,0.534483,0.111450,0.219531,Q06214,verified
1048,YPR179C,HDA3,656,512,577,GETLSKLKDAFVKTDNVQDEIEKEERVSVSRDTEKKYMEQEIKRAV...,66,0.012219,0.469697,0.045506,0.106750,Q06623,verified
1049,YPR186C,PZF1,430,370,411,NCSRTFKTKEKYEKHIDKHKVHELKLKILQEKEENKTLVDQN,42,0.011278,0.452381,0.031155,0.081428,P39933,verified


In [15]:
df[df.gene=='NOP14'].to_csv('hits_nop14.csv', index=False)

In [14]:
for index,row in df[df.gene=='NOP14'].iterrows():
    print(row['left.bound'], row['right.bound'], row['region.seq'])

19 74 GQTNVKSKNKKNSKRQAKEYDREEKKKAIAEIREEFNPFEIKAARNKRRDGLPSKT
85 144 ISKQIGEEQRKRAFEARKMMKNKRGGVIDKRFGERDKLLTEEEKMLERFTRERQSQSKRN
169 217 QSLSLEDELANDEEDFLASKRFNEDDAELQQPQRKKTKAEVMKEVIAKS
254 352 PKKNPMEPKTDLDKEYDIKVKELQLDKRAAPSDRTKTEEEKNAEAEEKKRELEQQRLDRMNGMIELEEGEERGVEDLDDGFWENEEDYEDDNDGIADSD
711 779 THAPKYEENFNPDKKSYDPDRTRSEINKMKAQLKKERKFTMKEIRKDAKFEARQRIEEKNKESSDYHAK


In [13]:
def append_orf_label(row):
    orf = row['orf']
    if orf in verified_orfs:
        return 'verified'
    elif orf in unchar_orfs:
        return 'uncharacterized'
    elif orf in dubious_orfs:
        return 'dubious'
    else:
        return None

In [14]:
df['orf_label'] = df.apply(lambda row: append_orf_label(row), axis=1)

In [15]:
#df.to_csv('../charged-regions-output/charged-regions-medthreshold-raw.csv', index=False)

In [16]:
bad_orfs_index = []
rv = []
for index, row in df.iterrows():
    if row['orf'] in bad_orfs:
        bad_orfs_index.append(index)
        rv.append(row['orf'])

In [17]:
len(np.unique(rv))

33

In [18]:
df.drop(df.index[bad_orfs_index], inplace=True)
df = df.reset_index(drop=True)

In [19]:
df

,orf,gene,seq.len,left.bound,right.bound,region.seq,region.len,charge.asymmetry,frac.charge,kappa1,kappa2,uni_id,orf_label
0,YAL011W,SWC3,626,0,54,MPAVLRTRSKESSIEQKPASRTRTRSRRGKRGRDDDDDDDDEESDD...,55,0.015674,0.527273,0.297167,0.616928,P31376,verified
1,YAL011W,SWC3,626,169,265,RLFILKNDKIEQKWQDEQELKKKEKELKRKNDAEAKRLRMEERKRQ...,97,0.050753,0.536082,0.051716,0.110243,P31376,verified
2,YAL011W,SWC3,626,361,424,KTAATEPEPKKADDENAEKQQSKEAKTTAESTQVDVKKEEEDVKEK...,64,0.007812,0.500000,0.044964,0.096912,P31376,verified
3,YAL011W,SWC3,626,470,525,KSVVEFLEDTDEIIISWIVIHNSKEIEKFKTKKIKAKLKADQKLNK...,56,0.000000,0.428571,0.114622,0.300249,P31376,verified
4,YAL013W,DEP1,406,81,163,TLTESLKRPHEDEKEAIDEAKKMKVPGENEDESKEEEKSQELEEAI...,83,0.141633,0.542169,0.093375,0.239955,P31385,verified
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1011,YPR163C,TIF3,437,379,433,SVYDVLRTEDDDEDEEAEKQNGDAKENKVDAAVEKLQDKTAQLTVE...,55,0.122909,0.454545,0.115449,0.366767,P34167,verified
1012,YPR169W,JIP5,493,434,491,LVGLSKEELLDELDKDLKEDHQEEKESNSKSVKKRKIMKENNKKKD...,58,0.000556,0.534483,0.111450,0.219531,Q06214,verified
1013,YPR179C,HDA3,656,512,577,GETLSKLKDAFVKTDNVQDEIEKEERVSVSRDTEKKYMEQEIKRAV...,66,0.012219,0.469697,0.045506,0.106750,Q06623,verified
1014,YPR186C,PZF1,430,370,411,NCSRTFKTKEKYEKHIDKHKVHELKLKILQEKEENKTLVDQN,42,0.011278,0.452381,0.031155,0.081428,P39933,verified


## Extracting regions with a valid AYbRAH MSA

In [20]:
all_orfs = pd.read_table('../../data/sc_orfs/yeast-all-orfs.txt', header=None, names=['orf'])

In [21]:
no_valid_msa = []
fdir = '/mnt/d/research/drummond-lab/data/aybrah-all/'
for index, row in df.iterrows():
    try:
        orf = row['orf']
        msa = AlignIO.read(open(fdir+str(orf)+'-aybrah.fa'), "fasta")
        if len(msa) <= 2:
            no_valid_msa.append(index)
    except:
        no_valid_msa.append(index)

In [22]:
len(no_valid_msa)

64

In [23]:
df.drop(df.index[no_valid_msa], inplace=True)
df = df.reset_index(drop=True)

In [24]:
df

,orf,gene,seq.len,left.bound,right.bound,region.seq,region.len,charge.asymmetry,frac.charge,kappa1,kappa2,uni_id,orf_label
0,YAL011W,SWC3,626,0,54,MPAVLRTRSKESSIEQKPASRTRTRSRRGKRGRDDDDDDDDEESDD...,55,0.015674,0.527273,0.297167,0.616928,P31376,verified
1,YAL011W,SWC3,626,169,265,RLFILKNDKIEQKWQDEQELKKKEKELKRKNDAEAKRLRMEERKRQ...,97,0.050753,0.536082,0.051716,0.110243,P31376,verified
2,YAL011W,SWC3,626,361,424,KTAATEPEPKKADDENAEKQQSKEAKTTAESTQVDVKKEEEDVKEK...,64,0.007812,0.500000,0.044964,0.096912,P31376,verified
3,YAL011W,SWC3,626,470,525,KSVVEFLEDTDEIIISWIVIHNSKEIEKFKTKKIKAKLKADQKLNK...,56,0.000000,0.428571,0.114622,0.300249,P31376,verified
4,YAL013W,DEP1,406,81,163,TLTESLKRPHEDEKEAIDEAKKMKVPGENEDESKEEEKSQELEEAI...,83,0.141633,0.542169,0.093375,0.239955,P31385,verified
...,...,...,...,...,...,...,...,...,...,...,...,...,...
947,YPR163C,TIF3,437,379,433,SVYDVLRTEDDDEDEEAEKQNGDAKENKVDAAVEKLQDKTAQLTVE...,55,0.122909,0.454545,0.115449,0.366767,P34167,verified
948,YPR169W,JIP5,493,434,491,LVGLSKEELLDELDKDLKEDHQEEKESNSKSVKKRKIMKENNKKKD...,58,0.000556,0.534483,0.111450,0.219531,Q06214,verified
949,YPR179C,HDA3,656,512,577,GETLSKLKDAFVKTDNVQDEIEKEERVSVSRDTEKKYMEQEIKRAV...,66,0.012219,0.469697,0.045506,0.106750,Q06623,verified
950,YPR186C,PZF1,430,370,411,NCSRTFKTKEKYEKHIDKHKVHELKLKILQEKEENKTLVDQN,42,0.011278,0.452381,0.031155,0.081428,P39933,verified


## Removing regions with the wrong SC sequence

In [25]:
aybrah_path = '/mnt/d/research/drummond-lab/data/aybrah-all/'
wrong_seq = []
for index, row in df.iterrows():
    orf = row['orf']
    msa = AlignIO.read(open(aybrah_path+str(orf)+'-aybrah.fa'), "fasta")
    contains_seq = False
    for record in msa:
        seq = crutil.remove_gaps(record.seq)
        if row['region.seq'] in seq:
            contains_seq = True
    if not contains_seq:
        wrong_seq.append(index)

In [26]:
df.drop(df.index[wrong_seq], inplace=True)
df = df.reset_index(drop=True)

In [27]:
df

,orf,gene,seq.len,left.bound,right.bound,region.seq,region.len,charge.asymmetry,frac.charge,kappa1,kappa2,uni_id,orf_label
0,YAL011W,SWC3,626,0,54,MPAVLRTRSKESSIEQKPASRTRTRSRRGKRGRDDDDDDDDEESDD...,55,0.015674,0.527273,0.297167,0.616928,P31376,verified
1,YAL011W,SWC3,626,169,265,RLFILKNDKIEQKWQDEQELKKKEKELKRKNDAEAKRLRMEERKRQ...,97,0.050753,0.536082,0.051716,0.110243,P31376,verified
2,YAL011W,SWC3,626,361,424,KTAATEPEPKKADDENAEKQQSKEAKTTAESTQVDVKKEEEDVKEK...,64,0.007812,0.500000,0.044964,0.096912,P31376,verified
3,YAL011W,SWC3,626,470,525,KSVVEFLEDTDEIIISWIVIHNSKEIEKFKTKKIKAKLKADQKLNK...,56,0.000000,0.428571,0.114622,0.300249,P31376,verified
4,YAL013W,DEP1,406,81,163,TLTESLKRPHEDEKEAIDEAKKMKVPGENEDESKEEEKSQELEEAI...,83,0.141633,0.542169,0.093375,0.239955,P31385,verified
...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,YPR163C,TIF3,437,379,433,SVYDVLRTEDDDEDEEAEKQNGDAKENKVDAAVEKLQDKTAQLTVE...,55,0.122909,0.454545,0.115449,0.366767,P34167,verified
935,YPR169W,JIP5,493,434,491,LVGLSKEELLDELDKDLKEDHQEEKESNSKSVKKRKIMKENNKKKD...,58,0.000556,0.534483,0.111450,0.219531,Q06214,verified
936,YPR179C,HDA3,656,512,577,GETLSKLKDAFVKTDNVQDEIEKEERVSVSRDTEKKYMEQEIKRAV...,66,0.012219,0.469697,0.045506,0.106750,Q06623,verified
937,YPR186C,PZF1,430,370,411,NCSRTFKTKEKYEKHIDKHKVHELKLKILQEKEENKTLVDQN,42,0.011278,0.452381,0.031155,0.081428,P39933,verified


In [28]:
df[df.orf == 'YCL011C']

,orf,gene,seq.len,left.bound,right.bound,region.seq,region.len,charge.asymmetry,frac.charge,kappa1,kappa2,uni_id,orf_label


In [29]:
df.to_csv('../../data/charged_regions/cr_filtered.csv', index=False)

## Optional clean-up: Remove MSAs with fewer than 10 long sequences

In [30]:
small_msas = []
for index, row in df.iterrows():
    orf = row['orf']
    msa = AlignIO.read(open(aybrah_path+str(orf)+'-aybrah.fa'), "fasta")
    n_long = 0
    long_ids = []
    for record in msa:
        seq = crutil.remove_gaps(record.seq)
        if (len(seq) >= 25) and (record.id not in long_ids): 
            n_long += 1
            long_ids.append(record.id)
    if n_long < 10:
        small_msas.append(index)

In [31]:
df.drop(df.index[small_msas], inplace=True)
df = df.reset_index(drop=True)

In [32]:
df

,orf,gene,seq.len,left.bound,right.bound,region.seq,region.len,charge.asymmetry,frac.charge,kappa1,kappa2,uni_id,orf_label
0,YAL011W,SWC3,626,0,54,MPAVLRTRSKESSIEQKPASRTRTRSRRGKRGRDDDDDDDDEESDD...,55,0.015674,0.527273,0.297167,0.616928,P31376,verified
1,YAL011W,SWC3,626,169,265,RLFILKNDKIEQKWQDEQELKKKEKELKRKNDAEAKRLRMEERKRQ...,97,0.050753,0.536082,0.051716,0.110243,P31376,verified
2,YAL011W,SWC3,626,361,424,KTAATEPEPKKADDENAEKQQSKEAKTTAESTQVDVKKEEEDVKEK...,64,0.007812,0.500000,0.044964,0.096912,P31376,verified
3,YAL011W,SWC3,626,470,525,KSVVEFLEDTDEIIISWIVIHNSKEIEKFKTKKIKAKLKADQKLNK...,56,0.000000,0.428571,0.114622,0.300249,P31376,verified
4,YAL013W,DEP1,406,81,163,TLTESLKRPHEDEKEAIDEAKKMKVPGENEDESKEEEKSQELEEAI...,83,0.141633,0.542169,0.093375,0.239955,P31385,verified
...,...,...,...,...,...,...,...,...,...,...,...,...,...
802,YPR163C,TIF3,437,379,433,SVYDVLRTEDDDEDEEAEKQNGDAKENKVDAAVEKLQDKTAQLTVE...,55,0.122909,0.454545,0.115449,0.366767,P34167,verified
803,YPR169W,JIP5,493,434,491,LVGLSKEELLDELDKDLKEDHQEEKESNSKSVKKRKIMKENNKKKD...,58,0.000556,0.534483,0.111450,0.219531,Q06214,verified
804,YPR179C,HDA3,656,512,577,GETLSKLKDAFVKTDNVQDEIEKEERVSVSRDTEKKYMEQEIKRAV...,66,0.012219,0.469697,0.045506,0.106750,Q06623,verified
805,YPR186C,PZF1,430,370,411,NCSRTFKTKEKYEKHIDKHKVHELKLKILQEKEENKTLVDQN,42,0.011278,0.452381,0.031155,0.081428,P39933,verified
